In [41]:
from geopy.distance import distance
import geojson
from shapely.geometry import shape
from shapely.geometry import Point


In [42]:
def get_pol(path):
    with open(path,'r') as file:
        gj = geojson.load(file)
    features = gj['features'][0]
    geom = features['geometry']
    return shape(geom)

In [43]:
pol = get_pol('Stockholm.geojson')
bounds = pol.bounds
min_lng, min_lat, max_lng, max_lat = bounds
ne = max_lat, max_lng
sw = min_lat, min_lng

In [44]:
def partition_lng(maxLat,minLat,nPartitions):
    delta = maxLat - minLat
    partitionDelta = delta/nPartitions

    upperBoundLats = [minLat + (i+1)*partitionDelta for i in range(nPartitions)]
    lowerBoundLats = [minLat + i*partitionDelta for i in range(nPartitions)]

    return upperBoundLats, lowerBoundLats

In [47]:
delta_lng_meters = distance((max_lat,max_lng),(max_lat,min_lng)).meters

In [48]:
delta_lng_degree = max_lng-min_lng

In [49]:
def get_lats_lngs_for_a_grid(max_lat, min_lat, max_lng, min_lng, meters_between_points=10):
    delta_lat_degree = max_lat-min_lat
    delta_lat_meters = distance((max_lat,max_lng),(min_lat,max_lng)).meters
    number_of_lat_partitions = int(delta_lat_meters/meters_between_points)
    delta_lat_degree_per_partition = delta_lat_degree/number_of_lat_partitions


    lats_for_the_grid = [min_lat + i * delta_lat_degree_per_partition
                        for i 
                        in range(number_of_lat_partitions)]

    delta_lng_degree = max_lng-min_lng
    delta_lng_meters = distance((max_lat,max_lng),(max_lat,min_lng)).meters
    number_of_lng_partitions = int(delta_lng_meters/meters_between_points)
    delta_lng_degree_per_partition = delta_lng_degree / number_of_lng_partitions

    lngs_for_the_grid = [min_lng + i * delta_lng_degree_per_partition
                        for i 
                        in range(number_of_lng_partitions)]
    
    return lats_for_the_grid, lngs_for_the_grid

In [50]:
meters_between_points = 10
lats_for_the_grid, lngs_for_the_grid = get_lats_lngs_for_a_grid(max_lat, min_lat, max_lng, min_lng, meters_between_points)

In [82]:
full_grid = [Point(j,i) for j in lngs_for_the_grid for i in lats_for_the_grid]

In [83]:
len(full_grid)

852240

In [84]:
grid_actual = [p for p in full_grid if pol.contains(p)]

In [85]:
len(grid_actual)

287367

In [93]:
lats = [p.y for p in grid_actual]
lngs = [p.x for p in grid_actual]

In [96]:
all_valid_points = list(zip(lats,lngs))

In [98]:
import pandas as pd
df = pd.DataFrame()

In [99]:
df['lats'] = lats
df['lngs'] = lngs
df['points'] = all_valid_points

In [100]:
df.to_csv('All_GridPoints_in_GeoJSON.csv',index=False)